# Data Engineer Certification - Practical Exam - Supplement Experiments

1001-Experiments makes personalized supplements tailored to individual health needs.

1001-Experiments aims to enhance personal health by using data from wearable devices and health apps.

This data, combined with user feedback and habits, is used to analyze and refine the effectiveness of the supplements provided to the user through multiple small experiments.

The data engineering team at 1001-Experiments plays a crucial role in ensuring the collected health and activity data from thousands of users is accurately organized and integrated with the data from supplement usage. 

This integration helps 1001-Experiments provide more targeted health and wellness recommendations and improve supplement formulations.


## Task

1001-Experiments currently has the following four datasets with four months of data:
 - "user_health_data.csv" which logs daily health metrics, habits and data from wearable devices,
 - "supplement_usage.csv" which records details on supplement intake per user,
 - "experiments.csv" which contains metadata on experiments, and
 - "user_profiles.csv" which contains demographic and contact information of the users.

Each dataset contains unique identifiers for users and/or their supplement regimen.

The developers and data scientsits currently manage code that cross-references all of these data sources separately, which is cumbersome and error-prone.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset.

The final dataset should provide a comprehensive view of each user's health metrics, supplement usage, and demographic information.

- To test your code, your manager will run only the code `merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')`
- Your `merge_all_data` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.


## Data

The provided data is structured as follows:

![database schema](schema.png)

The function you write should return data as described below.

There should be a unique row for each daily entry combining health metrics and supplement usage.

Where missing values are permitted, they should be in the default Python format unless stated otherwise.

| Column Name        | Description |
|--------------------|-------------|
| user_id            | Unique identifier for each user. </br>There should not be any missing values. |
| date               | The date the health data was recorded or the supplement was taken, in date format. </br>There should not be any missing values. |
| email              | Contact email of the user. </br>There should not be any missing values. |
| user_age_group  | The age group of the user, one of: 'Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65' or 'Unknown' where the age is missing.|
| experiment_name    | Name of the experiment associated with the supplement usage. </br>Missing values for users that have user health data only is permitted. |
| supplement_name    | The name of the supplement taken on that day. Multiple entries are permitted. </br>Days without supplement intake should be encoded as 'No intake'. |
| dosage_grams       | The dosage of the supplement taken in grams. Where the dosage is recorded in mg it should be converted by division by 1000.</br>Missing values for days without supplement intake are permitted. |
| is_placebo         | Indicator if the supplement was a placebo (true/false). </br>Missing values for days without supplement intake are permitted. |
| average_heart_rate | Average heart rate as recorded by the wearable device. </br>Missing values are permitted. |
| average_glucose    | Average glucose levels as recorded on the wearable device. </br>Missing values are permitted. |
| sleep_hours        | Total sleep in hours for the night preceding the current day’s log. </br>Missing values are permitted. |
| activity_level     | Activity level score between 0-100. </br>Missing values are permitted. |

In [122]:
import pandas as pd
import numpy as np

In [123]:
def merge_all_data(
    health_csv = 'user_health_data.csv', 
    supplement_csv = 'supplement_usage.csv', 
    experiment_csv = 'experiments.csv', 
    user_csv = 'user_profiles.csv'
):
    experiments = pd.read_csv(experiment_csv).rename(columns={'name': 'experiment_name'})
    supplement = pd.read_csv(supplement_csv, parse_dates=['date'])
    supplement['dosage_grams'] = supplement['dosage']/1000
    
    health = pd.read_csv(health_csv, parse_dates=['date'])
    health['sleep_hours'] = health['sleep_hours'].str.replace("(?i)h$", "", regex=True).astype(float)
    
    user = pd.read_csv(user_csv)
    
    bins = [0, 17, 25, 35, 45, 55, 65, 70]
    ages = ['Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65']
    user['user_age_group'] = pd.cut(user['age'], bins=bins, labels=ages).cat.add_categories(['Unknown']).fillna('Unknown')
    
    supplement_cols = ['user_id', 'date', 'supplement_name', 'dosage_grams', 'is_placebo', 'experiment_id']
    experiment_cols = ['experiment_id', 'experiment_name']
    supp_exp =  supplement[supplement_cols].merge(experiments[experiment_cols], on='experiment_id', how='left')
    
    user_cols = ['user_id', 'email', 'user_age_group']
    supp_exp_cols = ['user_id', 'date', 'supplement_name', 'dosage_grams', 'is_placebo', 'experiment_id', 'experiment_name']
    supp_exp_user = supp_exp[supp_exp_cols].merge(user[user_cols], on='user_id', how='left')
    supp_exp_user_cols = ['user_id', 'date', 'email', 'user_age_group', 'experiment_name', 'supplement_name', 'dosage_grams', 'is_placebo']
    
    health_cols = ['user_id', 'date', 'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level']
    health_user = health[health_cols].merge(user[user_cols], on='user_id', how='left')
    health_user_cols = ['user_id', 'date', 'email', 'user_age_group', 'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level']
    
    same_dates = supp_exp_user[supp_exp_user_cols].merge(health_user[health_user_cols], on=['user_id', 'date', 'email', 'user_age_group'], how='outer')
    final_cols = ['user_id', 'date', 'email', 'user_age_group', 'experiment_name', 'supplement_name', 'dosage_grams', 'is_placebo', 'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level']
    same_dates['supplement_name'] = same_dates['supplement_name'].fillna('No intake')
    return same_dates[final_cols]

In [124]:
merge_all_data().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2721 entries, 0 to 2720
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_id             2721 non-null   object        
 1   date                2721 non-null   datetime64[ns]
 2   email               2721 non-null   object        
 3   user_age_group      2721 non-null   category      
 4   experiment_name     2000 non-null   object        
 5   supplement_name     2721 non-null   object        
 6   dosage_grams        2000 non-null   float64       
 7   is_placebo          2000 non-null   object        
 8   average_heart_rate  2721 non-null   float64       
 9   average_glucose     2721 non-null   float64       
 10  sleep_hours         2721 non-null   float64       
 11  activity_level      2721 non-null   int64         
dtypes: category(1), datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 258.1+ KB
